In [1]:
import pickle
from pathlib import Path
DATA_DIR = Path('')
def load(filename):
    f = open(DATA_DIR/filename,"rb")
    return pickle.load(f)
    
def save(data, filename):
    with open(DATA_DIR/filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
from aoc_utils import *
import tarfile
fname = 'synacor-challenge.tgz'
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
elif fname.endswith("tgz"):
    tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()


In [47]:
import numpy as np

f = open('challenge.bin', 'rb')
ins = np.fromfile(f, dtype=np.uint16)


class Comp():
    def __init__(self, ins, verbose=False):
        self.pnt = 0
        self.ins = ins
        self.functions = {
            0 : (self.halt,0,0),
            1 : (self.set,1,1),
            2 : (self.push,0,1),
            3 : (self.pop,1,0),
            4 : (self.eq,1,2),
            5 : (self.gt,1,2),
            6 : (self.jmp, 0,1),
            7 : (self.jt, 0,2),
            8 : (self.jf, 0,2),
            9 : (self.add, 1,2),
            10 : (self.mult, 1,2),
            11 : (self.mod, 1,2),
            12: (self.func_and, 1,2),
            13 : (self.func_or, 1,2),
            14 : (self.func_not, 1,1),
            15 : (self.rmem, 1,1),
            16 : (self.wmem, 0,2),
            17 : (self.call, 0,1),
            18 : (self.ret, 0,0),
            19: (self.out,0,1),
            20: (self.func_in,1,0),
            21: (self.noop,0,0),
        }
        self.M = 32768
        self.reg = {i:0 for i in range(8)}
        self.stack = []
        self.verbose = verbose
        self.in_loc = 0
        self.hunting = False

    def halt(self, args):
        #should not be called
        sys.exit()

    def set(self, args):
        self.reg[args[0]] = args[1]
        if self.verbose: print(f'reg {args[0]} is now {self.reg[args[0]]}')

    def push(self, args):
        #   push <a> onto the stack
        self.stack.append(args[0])
        if self.verbose: print('new stack', self.stack)

    def pop(self, args):
        top = self.stack.pop()
        self.reg[args[0]] = top
        if self.verbose: print('new stack', self.stack)

    def eq(self, args):
        #   set <a> to 1 if <b> is equal to <c>; set it to 0 otherwise
        a,b,c = args
        self.reg[a] = 1 if b == c else 0
        if self.verbose: print(f'reg {a} is now {self.reg[a]}')

    def gt(self, args):
        # set <a> to 1 if <b> is greater than <c>; set it to 0 otherwise
        a,b,c = args
        self.reg[a] = 1 if b > c else 0

    def jmp(self, args):
        return args[0]

    def jt(self, args):
        if args[0] != 0:
            return args[1]

    def jf(self, args):
        if args[0] == 0:
            return args[1]

    def add(self, args):
        # assign into <a> the sum of <b> and <c> (modulo 32768)
        a, b, c = args
        self.reg[a] = (b + c) % self.M

    def mult(self, args):
        # assign into <a> the product of <b> and <c> (modulo 32768)
        a, b, c = args
        self.reg[a] = (b * c) % self.M

    def mod(self, args):
        # store into <a> the remainder of <b> divided by <c>
        a, b, c = args
        self.reg[a] = (b % c) % self.M

    def func_and(self, args):
        # stores into <a> the bitwise and of <b> and <c>
        a, b, c = args
        self.reg[a] = (b & c) % self.M

    def func_or(self, args):
        # stores into <a> the bitwise or of <b> and <c>
        a, b, c = args
        self.reg[a] = (b | c) % self.M

    def func_not(self, args):
        # stores 15-bit bitwise inverse of <b> in <a>
        a, b = args
        self.reg[a] = (~ b) % self.M

    def rmem(self, args):
        #   read memory at address <b> and write it to <a>
        a , b = args
        self.reg[a] = self.ins[b]
        if self.verbose: print(f'written to reg {a} val {self.ins[b]} from pnt {b}')

    def wmem(self, args):
        #   write the value from <b> into memory at address <a>
        a , b = args
        self.ins[a] = b
        if self.verbose: print(f'written {b} to address {a}')

    def call(self, args): 
        if self.verbose: print(self.pnt)
        self.stack.append(self.pnt)
        return args[0]

    def ret(self, args):
        # remove the top element from the stack and jump to it; empty stack = halt
        if self.verbose: print('retting')
        
        return self.stack.pop()
        # sys.exit()

    def out(self,args):
        print(chr(args[0]), end='')

    def func_in(self, args):
        self.reg[self.in_loc] = args
        self.run()  

    def receive_input(self, args):
        self.reg[self.in_loc] = args
        return self.run()  

    def noop(self,args):
        pass


    def do_function(self):
        
        # read opcode
        if self.verbose: print('pointer', self.pnt, '\nregisters', self.reg, '\nstack', self.stack)
        opcode = self.ins[self.pnt]
        self.pnt += 1
        func, num_write, num_read = self.functions[opcode]
        if func == self.halt:
            print('halting')
            sys.exit()
            return False
        # read arguments
        if self.verbose: print('arguments from addresses',[self.ins[self.pnt+i] for i in range(num_write+num_read)])
        # if 7+self.M in [self.ins[self.pnt+i] for i in range(num_write+num_read)]:
        #     print('found mtf')
        #     self.verbose = True
        #     self.hunting = 20
        # if self.hunting > 0 : self.hunting -= 1
        # if self.hunting == 0:
        #     self.verbose=False
        args = tuple(self.ins[self.pnt+i] % self.M for i in range(num_write))
        self.pnt += num_write
        args += tuple(self.ins[self.pnt+i] if self.ins[self.pnt+i] < 32768 else self.reg[self.ins[self.pnt+i] % self.M] for i in range(num_read))
        self.pnt += num_read
        if self.verbose: print('................',func.__name__, args)

        if func == self.func_in:
            print('waiting for input')
            self.in_loc = args[0]
            return False

        # optional: change pointer
        res = func(args)
        if res: # could do this with walrus
            if self.verbose: print('new pointer', res)
            self.pnt = res
        if res == 0:
            print('res 0')
            sys.exit()
        
        # print(opcode, args, func, numargs, pnt)
        return True
    
    def run(self,amount=1000000):
        for i in range(amount):
            # if i == 10751: self.reg[5]=0
            # if i == 10753: self.reg[3]=0
            # if i > 10279: self.verbose=True
            # if i > 11279: self.verbose=False
            if self.verbose: print('\n',i,self.pnt)
            res = self.do_function()
            if not res:
                # print(i)
                return
c = load('readyforteleport.pickle')
new = Comp({i:val for i, val in enumerate(list(ins))},verbose=False)
new.reg = c.reg
new.ins = c.ins
new.pnt = c.pnt
new.in_loc = c.in_loc
new.stack = c.stack

In [165]:
def send_text(text):
    if text == 'n' : text = 'north'
    if text == 's' : text = 'south'
    if text == 'e' : text = 'east'
    if text == 'w' : text = 'west'
    for chr in text:
        new.receive_input(ord(chr))
    new.receive_input(ord('\n'))
def send_multiple(commands):
    for text in commands:
        send_text(text)
# new.reg[7]=5
# new.ins[6035]=7
new.verbose=False

# new = load('pedestal.pickle')
send_text('use mirror')
# send_multiple('neenwseewnne')
neenwseewnne

waiting for input
waiting for input
waiting for input
waiting for input
waiting for input
waiting for input
waiting for input
waiting for input
waiting for input
waiting for input


You gaze into the mirror, and you see yourself gazing back.  But wait!  It looks like someone wrote on your face while you were unconscious on the beach!  Through the mirror, you see "uMipOAHxMdwx" scrawled in charcoal on your forehead.

Congratulations; you have reached the end of the challenge!


What do you do?
waiting for input


In [197]:
# option 1 to solve
cell2val = {2:4,3:4,4:8,5:9,6:11,7:18,8:1}
from aoc_utils import *
from collections import namedtuple
from operator import mul, sub, add
conn = {
    1: ((add,2) , (add, 3), (sub, 3), (sub, 5)),
    2: ((mul, 4),(mul,6),(mul,3), (add,3)),
    3: ((add,2), (mul,2), (mul,4),(mul,6), (sub,6),(sub,5),(sub,7)),
    4: ((mul,2),(mul,3),(mul,6),(sub,6),(sub,8)),
    5: ((sub,3),(sub,7),(mul,7),(sub,6)),
    6: ((mul,2),(mul,3),(sub,3),(mul,4),(sub,4),(sub,5),(sub,7),(mul,7),(mul,8),(sub,8)),
    7: ((sub,3),(sub,5),(mul,5),(sub,6),(mul,6),(mul,8)),
    8: ((sub,4),(sub,6),(mul,6),(mul,7))
}
state = namedtuple('state', ['cell','val'])
options = set()
def getneigh(s):
    for c in conn[s.cell]:
        res = state(c[1], c[0](s.val,cell2val[c[1]]))
        if 0<res.val<100: options.add(res)
    return options
s = state(1,22)
getneigh(s)
bfs(getneigh, start = state(1,22),goal=state(8,30))

[state(cell=1, val=22),
 state(cell=3, val=26),
 state(cell=6, val=15),
 state(cell=3, val=60),
 state(cell=7, val=42),
 state(cell=6, val=31),
 state(cell=8, val=30)]

In [195]:
# option 2 to solve. Easier to code and gives answer fully. Bit more code though
lines = [[mul,8,sub,1],[4,mul,11,mul],[add,4,sub,18],[22,sub,9,mul]]
valid = {(row,col) for row in range(len(lines)) for col in range(len(lines[0]))}
valid.remove((3,0))

from collections import namedtuple
state = namedtuple('state', ['cell','val'])
s = state((3,0),22)
dr = [0,1,0,-1]
dc = [1,0,-1,0]
def getneigh(s):
    r,c = s.cell
    options = set()

    for i in range(4):
        newr = r+dr[i]
        newc = c+dc[i]
        if (newr,newc) in valid:
            if isinstance(lines[newr][newc], int):
                op = lines[r][c]
                # print(op)
                options.add(state((newr,newc),op(s.val,lines[newr][newc])))
            else:
                options.add(state((newr,newc),s.val))
    return options
states = bfs(getneigh, start=s, goal=state((0,3),30))
ans = []

for comb in zippify(states,2):
    dr = comb[1].cell[0] - comb[0].cell[0]
    dc = comb[1].cell[1] - comb[0].cell[1]
    if dr == -1: ans.append('n')
    if dr ==  1: ans.append('s')
    if dc ==  1: ans.append('e')
    if dc == -1: ans.append('w')
print(''.join(ans))

neenwseewnne


In [86]:
qamHPfkEUpxt
qamHPfkEUpxt
HjbymqFNVYsE
HjbymqFNVYsE
rnneuyhJRoco
HjbymqFNVYsE

7

In [71]:
save(c,'pedestal.pickle')

In [30]:
c = load('readyforteleport.pickle')

In [90]:
Fireflies were using this dusty old journal as a resting spot until you scared them off.  It reads:

Day 1: We have reached what seems to be the final in a series of puzzles guarding an ancient treasure.  I suspect most adventurers give up long before this point, but we're so close!  We must press on!

Day 1: P.S.: It's a good thing the island is tropical.  We should have food for weeks!

Day 2: The vault appears to be sealed by a mysterious force - the door won't budge an inch.  We don't have the resources to blow it open, and I wouldn't risk damaging the contents even if we did.  We'll have to figure out the lock mechanism.

Day 3: The door to the vault has a number carved into it.  Each room leading up to the vault has more numbers or symbols embedded in mosaics in the floors.  We even found a strange glass orb in the antechamber on a pedestal itself labeled with a number.  What could they mean?

Day 5: We finally built up the courage to touch the strange orb in the antechamber.  It flashes colors as we carry it from room to room, and sometimes the symbols in the rooms flash colors as well.  It simply evaporates if we try to leave with it, but another appears on the pedestal in the antechamber shortly thereafter.  It also seems to do this even when we return with it to the antechamber from the other rooms.

Day 8: When the orb is carried to the vault door, the numbers on the door flash black, and then the orb evaporates.  Did we do something wrong?  Doesn't the door like us?  We also found a small hourglass near the door, endlessly running.  Is it waiting for something?

Day 13: Some of my crew swear the orb actually gets heaver or lighter as they walk around with it.  Is that even possible?  They say that if they walk through certain rooms repeatedly, they feel it getting lighter and lighter, but it eventually just evaporates and a new one appears as usual.

Day 21: Now I can feel the orb changing weight as I walk around.  It depends on the area - the change is very subtle in some places, but certainly more noticeable in others, especially when I walk into a room with a larger number or out of a room marked '*'.  Perhaps we can actually control the weight of this mysterious orb?

Day 34: One of the crewmembers was wandering the rooms today and claimed that the numbers on the door flashed white as he approached!  He said the door still didn't open, but he noticed that the hourglass had run out and flashed black.  When we went to check on it, it was still running like it always does.  Perhaps he is going mad?  If not, which do we need to appease: the door or the hourglass?  Both?

Day 55: The fireflies are getting suspicious.  One of them looked at me funny today and then flew off.  I think I saw another one blinking a little faster than usual.  Or was it a little slower?  We are getting better at controlling the weight of the orb, and we think that's what the numbers are all about.  The orb starts at the weight labeled on the pedestal, and goes down as we leave a room marked '-', up as we leave a room marked '+', and up even more as we leave a room marked '*'.  Entering rooms with larger numbers has a greater effect.

Day 89: Every once in a great while, one of the crewmembers has the same story: that the door flashes white, the hourglass had already run out, it flashes black, and the orb evaporates.  Are we too slow?  We can't seem to find a way to make the orb's weight match what the door wants before the hourglass runs out.  If only we could find a shorter route through the rooms...

Day 144: We are abandoning the mission.  None of us can work out the solution to the puzzle.  I will leave this journal here to help future adventurers, though I am not sure what help it will give.  Good luck!

What do you do?
waiting for input

8

In [259]:
from itertools import permutations
for comb in permutations([3,9,2,5,7]):
    if comb[0] + comb[1]*(comb[2]**2) + comb[3]**3 - comb[4] == 399:
        print (comb)
        break

(9, 2, 5, 7, 3)


In [7]:
c.M+8

32776

In [8]:
32776%c.M

8